# Autoencoder

- https://github.com/SherlockLiao/pytorch-beginner/tree/master/08-AutoEncoder

In [13]:
import os
import numpy as np
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

In [3]:
cuda = torch.cuda.is_available()
if cuda:
    print('cuda is available!')

In [4]:
num_epochs = 100
batch_size = 128
learning_rate = 0.001
out_dir = './autoencoder'

In [5]:
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [6]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # [0,1] => [-1,1]
])
train_dataset = MNIST('./data', transform=img_transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [7]:
print(len(train_dataset))
print(len(train_loader))

60000
469


In [8]:
class Autoencoder(nn.Module):
    
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True),
            nn.Linear(64, 12),
            nn.ReLU(True),
            nn.Linear(12, 3))
        
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128, 28 * 28),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

model = Autoencoder()
if cuda:
    model.cuda()

In [9]:
model

Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=128)
    (1): ReLU(inplace)
    (2): Linear(in_features=128, out_features=64)
    (3): ReLU(inplace)
    (4): Linear(in_features=64, out_features=12)
    (5): ReLU(inplace)
    (6): Linear(in_features=12, out_features=3)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12)
    (1): ReLU(inplace)
    (2): Linear(in_features=12, out_features=64)
    (3): ReLU(inplace)
    (4): Linear(in_features=64, out_features=128)
    (5): ReLU(inplace)
    (6): Linear(in_features=128, out_features=784)
    (7): Tanh()
  )
)

In [10]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=learning_rate,
                             weight_decay=1e-5)

In [11]:
def to_img(x):
    x = 0.5 * (x + 1)  # [-1,1] => [0, 1]
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [23]:
loss_list = []

for epoch in range(num_epochs):
    for data in train_loader:
        img, _ = data
        img = img.view(img.size(0), -1)
        if cuda:
            img = Variable(img).cuda()
        else:
            img = Variable(img)
        
        output = model(img)
    
        # 出力画像（再構成画像）と入力画像の間でlossを計算
        loss = criterion(output, img)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # logging
        loss_list.append(loss.data[0])
    
    print('epoch [{}/{}], loss: {:.4f}'.format(
        epoch + 1,
        num_epochs,
        loss.data[0]))

    # 10エポックごとに再構成された画像（output）を描画する
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './{}/image_{}.png'.format(out_dir, epoch))

epoch [1/100], loss: 0.1213
epoch [2/100], loss: 0.1338
epoch [3/100], loss: 0.1229


KeyboardInterrupt: 

- バッチサイズ128なのに12x8=96枚の画像が出力される
- 60000 % 128 = 96なので最後のバッチでは96枚の画像だけなので正しい

In [14]:
np.save('./{}/loss_list.npy'.format(out_dir), np.array(loss_list))
torch.save(model.state_dict(), './{}/autoencoder.pth'.format(out_dir))

In [16]:
ls autoencoder/

autoencoder.pth  image_20.png     image_50.png     image_80.png
image_0.png      image_30.png     image_60.png     image_90.png
image_10.png     image_40.png     image_70.png     loss_list.npy
